In [19]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
import time
import os
import re
import pandas as pd
import numpy as np
import urllib
import unicodedata

from urllib.request import urlopen
from urllib.parse import quote_plus

import requests
import shutil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from pyvirtualdisplay import Display



In [13]:
from datetime import datetime
import pandas as pd

(datetime.now() - pd.DateOffset(days=1)).strptime('%y')

TypeError: strptime() takes exactly 3 positional arguments (1 given)

In [6]:
spark = SparkSession \
    .builder \
    .appName("multi") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","ojo_db") \
    .config("spark.mongodb.input.collection", "insta_img") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()
sc =spark.sparkContext

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument("--single-process")
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("disable-gpu")
# chrome_options.add_argument("lang=ko_KR") # 한국어!
# chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36")
path='/home/ubuntu/chromedriver'
driver = webdriver.Chrome(path, chrome_options=chrome_options)


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: use options instead of chrome_options


In [23]:
def insta_searching(word):
    url = 'https://www.instagram.com/' + str(word)
    return url

def select_first(driver):
    first=driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
    
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 사진
    try:
        imgUrl = soup.select_one('div.PdwC2 .KL4Bh').img['src']
    except:
        imgUrl = 'not-image'
    # 본문내용
    try:
        content_origin = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content_origin)
    except:
        content = ' '
    #해시태그    
    tags = re.findall(r'#[^\s#,\\]+', content)
    #작성일자   
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    unix = iso2unix(date)
    
    #좋아요
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]
    except:
        like = 0
    #위치
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
        
    data=[content, date, like, place, tags, unix, imgUrl]
    return data

#함수 정의 : 첫번째 게시물 클릭 후 다음 게시물 클릭
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
# helpers
import time

import pytz

from iso8601 import parse_date
from pytz import reference

UTC = pytz.timezone('UTC')
LOCALZONE = reference.LocalTimezone()

def iso2unix(timestamp):
    """
    Convert a UTC timestamp formatted in ISO 8601 into a UNIX timestamp
    """

    # use iso8601.parse_date to convert the timestamp into a datetime object.
    # set the timezone to the local timezone in order for the time module to
    # properly manipulate it with localtime() and gmtime()
    parsed = parse_date(timestamp).astimezone(LOCALZONE)

    # now grab a time tuple that we can feed mktime
    timetuple = parsed.timetuple()

    # return a unix timestamp
    return time.mktime(timetuple)

def main():
    path='../TIL/Data-Analysis/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get("https://www.instagram.com/")
    time.sleep(2) #로딩 대기

    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys('01087334255')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys('hope4255^^')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()

    word= input("검색어 입력 : ")
    word = str(word)
    url = insta_searching(word)
    driver.get(url)
    time.sleep(5)

    select_first(driver)

    results = []
    n=0

    target = 500
    for i in range(target):
        #get content and append in result arr
        data=get_content(driver)
    #     db.insta_img.insert_one({'unix':data[5],'imgUrl':data[6]})
        results.append(data)
        #save data
    #     url = data[-1]
    #     unix = data[-2]
    #     resp = requests.get(url, stream=True)
    #     local_file = open('./img/' + word+'/'+ str(unix) + '.jpg', 'wb')
    #     resp.raw.decode_content = True
    #     shutil.copyfileobj(resp.raw, local_file)
    #     del resp

        #move next
        move_next(driver)

    driver.close()

    results_df = pd.DataFrame(results)
    results_df.columns = ['content', 'date', 'like', 'place', 'tags', 'unix', 'imgUrl']
    # results_df.to_csv('./img/'+ word + '/' + word + '.csv', encoding='utf-8-sig')
    return results_df
    
# if __name__=='__main__':
#     main()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=93.0.4577.82)


In [59]:
# import os
# from glob import glob
# # os.getcwd()
# glob('*')
# print("join(): " + os.path.join(os.path.abspath(os.path.dirname('/')), "file.py"))

join(): C:\file.py


In [49]:
# 이미지는 src로 가져와야됨

#인스타그램 주소로 웹 오픈 




# driver.implicitly_wait(5)

# tt = driver.find_element_by_css_selector("html")
# driver.get_screenshot_as_file('capture.png')
# print(tt.text)


# path='/home/ubuntu/chromedriver'
# driver = webdriver.Chrome(path)

# driver.get('https://www.instagram.com/')
# time.sleep(5)

# id_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input")
# password_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(2) > div > label > input")
# login_button = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

#액션체인 정의 act = ActionChains(dr)
# act = ActionChains(driver)

# act.send_keys_to_element(id_box,'01053153004').send_keys_to_element(password_box,'7030136a').click(login_button).perform()
# time.sleep(5)

# 인스타그램 로그인을 위한 계정 정보
# email = '01053153004'
# input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
# input_id.clear()
# input_id.send_keys(email)

# password = '7030136a'
# input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]

# input_pw.clear()
# input_pw.send_keys(password)
# input_pw.submit()


# time.sleep(5)


# 게시물을 조회할 검색 키워드 입력 요청

# if os.path.isdir('./img/'+ word) == False:
#     os.mkdir('./img/'+ word)
url = insta_searching(word)

# 검색 결과 페이지 열기
#driver.get(url)
driver.get(url)
time.sleep(5)

#첫 번째 게시물 클릭

# tt = driver.find_element_by_css_selector("html")

# print(tt.text)

# select_first(driver)

# results = []


# html = driver.page_source
# soup = BeautifulSoup(html)

검색어 입력 :  hi.stay.tour


In [8]:
# # pymongo connect
# client = MongoClient('localhost',27017) # mongodb 27017 port
# db = client.ojo

n=0

target = 500
for i in range(target):
    #try:
    #get content and append in result arr
    data=get_content(driver)
    db.insta_img.insert_one({'unix':data[5],'imgUrl':data[6]})
    results.append(data)
    #save data
    url = data[-1]
    unix = data[-2]
    resp = requests.get(url, stream=True)
    local_file = open('./img/' + word+'/'+ str(unix) + '.jpg', 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    del resp

    #move next
    move_next(driver)
    #except:
        #time.sleep(2)
        #move_next(driver)
        
'''
    # This is the image url.
    image_url = imglist[n]
    # Open the url image, set stream to True, this will return the stream content.
    resp = requests.get(image_url, stream=True)
    # Open a local file with wb ( write binary ) permission.
    
    local_file = open('./img/' + urllist[i] + str(n) + '.jpg', 'wb')
    
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    n +=1
    del resp
'''
driver.close()

검색어 입력 : hi.stay.tour


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._9AhH0"}
  (Session info: chrome=93.0.4577.82)


In [ ]:
results_df = pd.DataFrame(res
                         )
results_df.columns = ['content', 'date', 'like', 'place', 'tags', 'unix', 'imgUrl']
results_df.to_csv('./img/'+ word + '/' + word + '.csv', encoding='utf-8-sig')


In [7]:
input_pw

<selenium.webdriver.remote.webelement.WebElement (session="084341e53a53f0fd7b6260a2e2726b87", element="df14bff3-0720-4d2a-b1db-9bd783975f37")>

In [8]:
input_id

<selenium.webdriver.remote.webelement.WebElement (session="084341e53a53f0fd7b6260a2e2726b87", element="087b2da9-82d6-40ee-9b39-1e3fc8b3d40e")>

In [39]:
a=pd.read_csv('tot_dataset.csv')

In [42]:
a

,Unnamed: 0,content,date,like,tags,name,overlap,place
0,0,"속초_파도의무늬오션뷰 감성 에어비앤비""파도의무늬""'파도의무늬'는물결이 일렁이고 윤슬...",2021-09-06,39,"['#데일리감성속초로', '#데일리감성_속초강원도', '#데일리감성_강원——————...",파도의무늬,1,NaN
1,1,"강릉_스테이 시호일시간의 흔적을 담은 감성숙소""스테이 시호일""'스테이 시호일'은60...",2021-09-03,12,"['#데일리감성강릉으로', '#데일리감성_강릉강원도', '#데일리감성_강원—————...",스테이 시호일,1,NaN
2,2,"제주_까만돌멩이구좌읍 조식주는 예쁜 숙소""까만돌멩이""'까만돌멩이'는가족 및 커플 또...",2021-09-03,33,"['#데일리감성제주도', '#데일리감성_제주—————————————————']",까만돌멩이,1,제주특별자치도 제주시 구좌읍 계룡길 32
3,3,"단양_권장사양인스타감성 가득한 독채 숙소""권장사양""'권장사양'은 자연경관으로 둘러쌓...",2021-09-02,36,"['#데일리감성단양으로', '#데일리감성_단양충청북도', '#데일리감성_충북————...",권장사양,1,NaN
4,4,"강릉_라온가따뜻한 느낌의 힐링 감성숙소""라온가""'라온가'는 주택을 개조한 숙소이며'...",2021-09-01,24,"['#데일리감성강릉으로', '#데일리감성_강릉강원도', '#데일리감성_강원—————...",라온가,1,NaN
...,...,...,...,...,...,...,...,...
604,604,".한 폭의 그림같은 오션뷰, ""제주_Salt”🌊✨액자 창문 너머로 보이는 바다만 보...",2021-06-24,142,"['#제주', '#솔트', '#제주숙소', '#제주여행', '#제주숙소추천', '#...",솔트,2,NaN
605,605,".통창 리버뷰가 환상적인 ""춘천_피그멜리온이펙트""리버+마운틴뷰를 보며 월풀스파를 즐...",2021-06-15,87,"['#춘천', '#피그멜리온이펙트', '#춘천숙소', '#춘천여행', '#춘천숙소추...",피그멜리온이펙트,2,강원도 춘천시 서면 경춘로 405
606,606,"작은 시골 동네 안에 위치한 오붓&편안한 감성숙소,곡성_품안의밤!🧡논밭뷰를 보며 여...",2021-04-21,68,"['#곡성', '#품안의밤', '#곡성숙소', '#곡성여행', '#곡성숙소추천', ...",품안의밤,2,NaN
607,607,"KBS2에서 방송됐던 ""한 번 다녀왔습니다""의 웨딩촬영지로 유명한 강릉_더브리앙뜨!...",2021-04-19,57,"['#강릉', '#더브리앙뜨', '#강릉숙소', '#강릉여행', '#강릉데이트', ...",더브리앙뜨,2,강원도 강릉시 사천면 중앙서로 92-43


In [50]:
a.iloc[604]['content']

'.한 폭의 그림같은 오션뷰, "제주_Salt”🌊✨액자 창문 너머로 보이는 바다만 보고 있어도 시간 가는 줄 모르는 감성숙소랍니돠ㅏㅏ미니멀리즘을 추구하는 올 화이트톤의 숙소로 혼자 여행하는 칭구들에게 더욱 추천함! ٩( ᐛ )@@저장해두고 꼭 가보쟈구🤙📍제주 솔트 ( @saltaewol ) 주소 : 제주 제주시 애월읍 애월해안로 226📌 입실시간 : PM 16:00 📌 퇴실시간 : AM 11:00✔다녀왔던 이쁜 숙소 사진에 @sookso.diary 태그해주세요 콘탠츠제작(사진활용)시 상품권 5,000원을 보내드립니다 :D (직접 찍으신 사진만 가능해요!) #제주 #솔트 #제주숙소 #제주여행#제주숙소추천 #솔트애월 #애월솔트#제주감성숙소 #제주오션뷰숙소#제주숙소추천 #제주호스텔#제주도 #솔트 바다를 옆에 두고 잠들 수 있는 제주 애월숙소, 솔트입니다. 솔트는 모든 객실이 1-2인실로 구성되어 있다고 합니다. 또한, 전객실이 바다를 바라보는 방법을 토대로 디자인되어 있다고 합니다. 바다를 가까이에 두고 잠들 수 있다는 것만으로도 이곳에 머무를만한 이유가 되지 않을까요? 이곳의 시그니처라고 불리는 룸타입은 작은 창 셋 싱글베드 둘로 구성된 공간입니다. 창의 높낮이가 달라 각자 다른 바다를 경험해볼 수 있습니다. 심플하면서도 아늑한 공간, 솔트애월에서 물멍하며 하루를 보내보시길 바래요 :) \U0001fa84이런 분들에게 추천해드려요✧#파도소리 들으며 잠들고, 일어나고 싶은 분✧#미니멀리즘 이 느껴지는 숙소를 찾는 분✧노을이 지는 #애월바다 의 풍경을 눈에 담고 싶은 분__ __ __ __ __ __ __ __ __ __ __ __ 주소: 제주 제주시 애월읍 애월해안로 226체크인: 15:00 / 체크아웃: 11:00 숙소문의: @saltaewol __ __ __ __ __ __ __ __ __ __ __ __ 💡여행지보다 숙소를 먼저 고르는 에디터가 추천해주는#숙소하다_제주도˳˳사진 출처: @sephzu @callme_by_heaven @mmin_yj @narae.